In [1]:
import os
import re
import csv
from pymystem3 import Mystem
m = Mystem()

In [23]:
agedict = {'T80': 20, 'T81': 20, 'T105': 22, 'T106': 22, 'T139': 24,'T154': 26, 'T156': 26,'T183': 28, 'T210': 30, 'T229': 32, 'Ya8': 20, 'Ya11': 22, 'Ya13': 24, 'Ya16': 26, 'Ya22': 28, 'Ya27': 30, 'Ya32': 32, 'L108': 20, 'L110': 22, 'L200': 24, 'L202': 26, 'L204': 28, 'L206': 30, 'L208': 32, 'vl108': 20, 'vl110': 22, 'vl200': 24, 'vl202': 26, 'vl204': 28, 'vl206': 30, 'vl208': 32, 'vi200': 24, 'vi202': 26, 'vi204': 28, 'vi206': 30, 'vi208': 32, 'rl108': 20, 'rl110': 22, 'rl200': 24, 'rl206': 30, 'rl208': 32}

files_li = os.listdir(r'DIRECTORY')#folder with .cha files

files = []

for i in files_li:
    if i.endswith('.cha'):
        files.append(i)


def get_id(filename):
    id = re.findall(r'([a-zA-Z]+)\d+new.cha', filename)[0]
    return id


def get_age(filename):
    d = re.findall(r'(\w+\d+)new.cha', filename)[0]
    age = agedict[d]
    return age

['T105new.cha', 'T106new.cha', 'T154new.cha', 'T156new.cha', 'T183new.cha', 'T229new.cha']


In [35]:
def cleanstring(string):
    if string.startswith('*CHI') or string.startswith('*CHX'):
            #print(li)
                    cleanli = string[6:]
                    no_spaces_in_bra = re.sub(r':{1,2} ', r'', cleanli)
                    no_coms = re.sub(r'@.+ ', r' ', no_spaces_in_bra)
                    no_punct = ''
                    if '[' in no_coms:
                        word_list = no_coms.split()
                        for word in word_list:
                            if word.startswith('['):
                                num = word_list.index(word)
                                del word_list[num - 1]
                        word_list = word_list[:-1]
                        no_coms = ' '.join(word_list)
                    else:
                        coml = no_coms.split()[:-1]
                        no_coms = ' '.join(coml)
                    for symb in no_coms:
                        if symb not in '.,;:?!()[]*':
                            no_punct += symb
                    return no_punct

In [36]:
for chaf in files:
    kidid = get_id(chaf)
    kidage = get_age(chaf)
    kidreplname = r'DIRECTORY_REPL' + kidid + str(kidage) + 'repl.csv'#new folder for files with repliques
    sf = open(chaf, 'r', encoding='utf-8')
    if os.path.exists(kidreplname) == True:
        kf = open(kidreplname, 'a', encoding='utf-8')
        file_writer = csv.writer(kf, delimiter = "\t", lineterminator="\r")
        #file_writer.writerow(["Child_ID", "Age (month)", "Rerl"])
        for l in sf:
            if l.startswith('*CHI') or l.startswith('*CHX'):
                clli = cleanstring(l)
                if clli != ' ':
                    file_writer.writerow([kidid, kidage, clli])
        kf.close()
    else:
        nkf = open(kidreplname, 'w', encoding='utf-8')
        file_writer = csv.writer(nkf, delimiter = "\t", lineterminator="\r")
        file_writer.writerow(["Child_ID", "Age (month)", "Repl"])
        for ll in sf:
            if ll.startswith('*CHI') or ll.startswith('*CHX'):
                cll = cleanstring(ll)
                if cll != ' ':
                    file_writer.writerow([kidid, kidage, cll])
        nkf.close()
    sf.close()

In [5]:
replsdir = os.listdir(r'DIRECTORY_REPL')
for ref in replsdir:
    ref1 = r'DIRECTORY_REPL' + ref
    short = ref[:-8] + '_'
    kidanname = r'DIRECTORY_WORDS' + short + 'analysis.csv'#new folder for word analysis
    rf = open(ref1, 'r', encoding='utf-8')
    file_reader = csv.DictReader(rf, delimiter = "\t")
    af = open(kidanname, 'w', encoding='utf-8')
    file_writer = csv.writer(af, delimiter = "\t", lineterminator="\r")
    file_writer.writerow(["Child_ID", "Age", "Repl", "Word", "POS", "Grammar"])

    for l in file_reader:
        repl = l['Repl']
        id = l['Child_ID']
        age = l['Age (month)']
        words = repl.split()#[:-1]
        for word in words:
            an = m.analyze(word)
            for one in an:
                if 'analysis' in one:
                    if len(one['analysis']) != 0:
                        grammar = one['analysis'][0]['gr']
                        pos = grammar.split('=')[0].split(',')[0]
                        file_writer.writerow([id, age, ' '.join(words), word, pos, grammar])
    
    rf.close()
    af.close()


KeyboardInterrupt: 

In [7]:
for ref in replsdir:
    ref1 = r'DIRECTORY_REPL' + ref
    short = ref[:-8] + '_'
    kidanname = r'DIRECTORY_REPLANALYSIS' + short + 'analysisreps.csv'#new folder for files with repliques analysis
    rf = open(ref1, 'r', encoding='utf-8')
    file_reader = csv.DictReader(rf, delimiter = "\t")
    af = open(kidanname, 'w', encoding='utf-8')
    file_writer = csv.writer(af, delimiter = "\t", lineterminator="\r")
    file_writer.writerow(["Child_ID", "Age", "Repl", "POS", "Grammar"])
    for line in file_reader:
#for repl in list_of_repls:
        repl = line['Repl']
        id = line['Child_ID']
        age = line['Age (month)']
        ws = repl.split()#[:1]
        crepl = ' '.join(ws)
        ana = m.analyze(crepl)
        gramlist = []
        poslist = []
        for ww in ana:
            if 'analysis' in ww:
                if len(ww['analysis']) != 0:
                    grammar = ww['analysis'][0]['gr']
                    pos = grammar.split('=')[0].split(',')[0]
                    poslist.append(pos)
                    gramlist.append(grammar)
        file_writer.writerow([id, age, crepl, ' '.join(poslist), ' '.join(gramlist)])

    rf.close()
    af.close()

    

In [17]:
final = r'FINAL_FILE'#file with final analysis
f2 = open(final, 'w', encoding='utf-8')
file_writer = csv.writer(f2, delimiter = "\t", lineterminator="\r")
file_writer.writerow(["Child_ID", "Age", "Param_Num", "Param_name", "Score", "Examples"])
#file_writer.writerow([name, age, 1, "Noun", param1_score, param1_list])

49

In [3]:
anfolder = os.listdir(r'DIRECTORY_WORDS')

for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param1_list = []
    param1_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if row['POS'] == 'S':
                if row['Word'] not in param1_list:
                    param1_list.append(row['Word'])
                    param1_score += 1
            if param1_score == 2:
                break
    file_writer.writerow([name, age, 1, "Noun", param1_score, param1_list])

In [4]:
#pronoun
for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param1_list = []
    param1_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if row['POS'] == 'SPRO':
                if row['Word'] not in param1_list:
                    param1_list.append(row['Word'])
                    param1_score += 1
            if param1_score == 2:
                break
    file_writer.writerow([name, age, 2, "Pronoun", param1_score, param1_list])

In [5]:
#modifiers
for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param1_list = []
    param1_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if row['POS'] == 'A' or row['POS'] == 'APRO' or row['POS'] == 'ANUM':
                if row['Word'] not in param1_list:
                    param1_list.append(row['Word'])
                    param1_score += 1
            if param1_score == 2:
                break
    file_writer.writerow([name, age, 3, "Modifiers", param1_score, param1_list])

In [7]:
repanlist = os.listdir(r'DIRECTORY_REPLANALYSIS')

for anf in repanlist:
    anf1 = r'DIRECTORY_REPLANALYSIS' + anf
    param3_list = []
    param3_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if 'A S' in row['POS'] or 'APRO S' in row['POS'] or 'ANUM S' in row['POS']:
                if row['Repl'] not in param3_list:
                    param3_list.append(row['Repl'])
                    param3_score += 1
            if param3_score == 2:
                break
    file_writer.writerow([name, age, 4, "Mod_Phrase", param3_score, param3_list])


#file_writer.writerow([name, age, 3, "Mod_Phrase", param3_score, param3_list])

In [8]:
#three word
for anf in repanlist:
    anf1 = r'DIRECTORY_REPLANALYSIS' + anf
    param3_list = []
    param3_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if 'A A' in row['POS'] or 'APRO A' in row['POS'] or 'ANUM A' in row['POS'] or 'APRO APRO' in row['POS'] or 'APRO ANUM' in row['POS'] or 'ANUM ANUM' in row['POS']:
                if row['Repl'] not in param3_list:
                    param3_list.append(row['Repl'])
                    param3_score += 1
            if param3_score == 2:
                break
    file_writer.writerow([name, age, 5, "Three_Phrase", param3_score, param3_list])

In [6]:
for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param2_list = []
    param2_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if row['POS'] == 'S' and 'мн' in row['Grammar']:
                if row['Word'] not in param2_list:
                    param2_list.append(row['Word'])
                    param2_score += 1
            if param2_score == 2:
                break
    file_writer.writerow([name, age, 7, "Number", param2_score, param2_list])

In [13]:
#upr
for anf in repanlist:
    anf1 = r'DIRECTORY_REPLANALYSIS' + anf
    param3_list = []
    param3_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if 'S PR S' in row['POS'] or ('S S' in row['POS'] and ('род' in row['Grammar'] or 'дат' in row['Grammar'])) or 'A PR S' in row['POS'] or 'NUM S' in row['POS'] or ('A S' in row['POS'] and ('род' in row['Grammar'] or 'дат' in row['Grammar'] or 'твор' in row['Grammar'])):
                if row['Repl'] not in param3_list:
                    param3_list.append(row['Repl'])
                    param3_score += 1
            if param3_score == 2:
                break
    file_writer.writerow([name, age, 8, "Comand", param3_score, param3_list])

In [15]:
#prim
for anf in repanlist:
    anf1 = r'DIRECTORY_REPLANALYSIS' + anf
    param3_list = []
    param3_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if ('S V' in row['POS'] and 'инф' in row['Grammar']) or ('S A' in row['POS'] and ('срав' in row['Grammar'])) or 'S ADV' in row['POS']:
                if row['Repl'] not in param3_list:
                    param3_list.append(row['Repl'])
                    param3_score += 1
            if param3_score == 2:
                break
    file_writer.writerow([name, age, 9, "Prim", param3_score, param3_list])

In [9]:
#prep word
for anf in repanlist:
    anf1 = r'DIRECTORY_REPLANALYSIS' + anf
    param3_list = []
    param3_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if 'PR S' in row['POS']:
                if row['Repl'] not in param3_list:
                    param3_list.append(row['Repl'])
                    param3_score += 1
            if param3_score == 2:
                break
    file_writer.writerow([name, age, 10, "Pred_Phrase", param3_score, param3_list])

In [10]:
#case
for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param2_list = []
    param2_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if (row['POS'] == 'S' or row['POS'] == 'A' or row['POS'] == 'ANUM' or row['POS'] == 'SPRO') and ('род' in row['Grammar'] or 'дат' in row['Grammar'] or 'твор' in row['Grammar'] or 'пр' in row['Grammar'] or 'вин' in row['Grammar']):
                if row['Word'] not in param2_list:
                    param2_list.append(row['Word'])
                    param2_score += 1
            if param2_score == 2:
                break
    file_writer.writerow([name, age, 11, "Case", param2_score, param2_list])

In [11]:
#rare forms
for anf in anfolder:
    anf1 = r'DIRECTORY_WORDS' + anf
    param2_list = []
    param2_score = 0
    with open(anf1, 'r', encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
        file_reader = csv.DictReader(r_file, delimiter = "\t")
        for row in file_reader:
            name = row['Child_ID']
            age = row["Age"]
            if ((row['POS'] == 'S') and ('парт' in row['Grammar'] or 'местн' in row['Grammar'])) or (('пут' in row['Word'] or 'брем' in row['Word'] or 'врем' in row['Word'] or 'стрем' in row['Word'] or 'им' in row['Word'] or 'плем' in row['Word']) and ('им' not in row['Grammar'])):
                if row['Word'] not in param2_list:
                    param2_list.append(row['Word'])
                    param2_score += 1
            if param2_score == 2:
                break
    file_writer.writerow([name, age, 12, "Rare forms", param2_score, param2_list])

In [16]:
f2.close()